# 达观长文本分类Baseline 模型
## 使用新的PANDAS api 加快读取大文本文件
使用iterator迭代器读取pandas
```
reader = pd.read_csv(f, sep=',', iterator=True)
reader.get_chunk(chunkSize)
```


## 基准模型
使用SVC对词袋统计TFIDF值进行分类

### read data

In [2]:
import pandas as pd, numpy as np

def gen_csv_feather(path,path_new):

    f = open(path)

    reader = pd.read_csv(f, sep=',', iterator=True)

    loop = True

    chunkSize = 10000

    chunks = []

    while loop:

        try:

            chunk = reader.get_chunk(chunkSize)

            chunks.append(chunk)

        except StopIteration:

            loop = False

            print("Iteration is stopped.")

    df = pd.concat(chunks, ignore_index=True)

    print(df.count())

    df.to_pickle(path_new)
    
%time gen_csv_feather("../data/train_set.csv","../data/train_set.pkl")

%time gen_csv_feather("../data/test_set.csv","../data/test_set.pkl")    

Iteration is stopped.
锘縤d         102277
article     102277
word_seg    102277
class       102277
dtype: int64
Iteration is stopped.
锘縤d         102277
article     102277
word_seg    102277
dtype: int64


In [6]:
import pandas as pd, numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn import svm

column = "word_seg"
train = pd.read_pickle('../data/train_set.pkl')
test = pd.read_pickle('../data/test_set.pkl')
train.columns = ['id','article','word_seg','class']
test.columns = ['id','article','word_seg']
train, validate = train_test_split(train, test_size=0.2)
test_id = test["id"].copy()
vec = TfidfVectorizer(ngram_range=(1,2),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
trn_term_doc = vec.fit_transform(train[column])
test_term_doc = vec.transform(test[column])
fid0=open('baseline.csv','w')

y=(train["class"]-1).astype(int)
lin_clf = svm.LinearSVC()
lin_clf.fit(trn_term_doc,y)
preds = lin_clf.predict(test_term_doc)
i=0
fid0.write("id,class"+"\n")
for item in preds:
    fid0.write(str(i)+","+str(item+1)+"\n")
    i=i+1
fid0.close()

In [7]:
from sklearn.metrics import accuracy_score
y_validate = (validate["class"]-1).astype(int)
trn_term_docccctrn_term_docc = vec.transform(validate[column])
test_term_doc = vec.transform(validate[column])
preds = lin_clf.predict(test_term_doc)

accu = accuracy_score(preds, y_validate)
print('准确率:%f'%accu)

准确率:0.789744


In [4]:
train.head()

,id,article,word_seg,class
0,0,7368 1252069 365865 755561 1044285 129532 1053...,816903 597526 520477 1179558 1033823 758724 63...,14
1,1,581131 165432 7368 957317 1197553 570900 33659...,90540 816903 441039 816903 569138 816903 10343...,3
2,2,7368 87936 40494 490286 856005 641588 145611 1...,816903 1012629 957974 1033823 328210 947200 65...,12
3,3,299237 760651 299237 887082 159592 556634 7489...,563568 1239563 680125 780219 782805 1033823 19...,13
4,4,7368 7368 7368 865510 7368 396966 995243 37685...,816903 816903 816903 139132 816903 312320 1103...,12


In [8]:
!head baseline.csv

id,class
0,1
1,4
2,13
3,4
4,15
5,1
6,15
7,19
8,3
